In [1]:
# Install Dependencies
#!pip install scikit-image
#!pip install scenedetect[opencv,progress_bar]

In [2]:
from __future__ import print_function
import io
import os

import pandas as pd
import numpy as np

from skimage import color
import cv2

import scenedetect
from scenedetect.video_manager import VideoManager
from scenedetect.scene_manager import SceneManager
from scenedetect.frame_timecode import FrameTimecode
from scenedetect.stats_manager import StatsManager
from scenedetect.detectors import ContentDetector

STATS_FILE_PATH = 'testvideo.stats.csv'
dataset_folder = "../../data/"
cuts_folder = "../../data/cuts/"

# Find video cuts

In [ ]:
def find_video_cuts(dataset_folder, cuts_folder):
    videos_to_detect = [
        os.path.join(dataset_folder, f)
        for f 
        in os.listdir(dataset_folder)
        if os.path.isfile(os.path.join(dataset_folder, f))
    ]
    for i, video_dir in enumerate(videos_to_detect):
        print(f"Finding cuts of Video {video_dir}: {i+1}/{len(videos_to_detect)}", end='\r')

        # Check for already existing files and skip if already calculated
        yt_id = vid_file.split('.')[0]
        vid_cut_file = f'{yt_id}_cuts.npy'
        vid_cut_dir = os.path.join(cuts_folder, vid_cut_file)
        if os.path.exists(vid_cut_dir):
            continue

        # Create a video_manager point to video file testvideo.mp4. Note that multiple
        # videos can be appended by simply specifying more file paths in the list
        # passed to the VideoManager constructor. Note that appending multiple videos
        # requires that they all have the same frame size, and optionally, framerate.
        try:
            video_manager = VideoManager([video_dir])
            stats_manager = StatsManager()
            scene_manager = SceneManager(stats_manager)
            # Add ContentDetector algorithm (constructor takes detector options like threshold).
            scene_manager.add_detector(ContentDetector())
            base_timecode = video_manager.get_base_timecode()

            try:
                # Set downscale factor to improve processing speed (no args means default).
                video_manager.set_downscale_factor()

                # Start video_manager.
                video_manager.start()

                # Perform scene detection on video_manager.
                scene_manager.detect_scenes(frame_source=video_manager)

                # Obtain list of detected scenes.
                scene_list = scene_manager.get_scene_list(base_timecode)
                # Like FrameTimecodes, each scene in the scene_list can be sorted if the
                # list of scenes becomes unsorted.

                times = np.array([s[0].get_frames() for s in scene_list])
                np.save(vid_cut_dir, times)
            finally:
                video_manager.release()
        except:
            pass

In [ ]:
# Find Cinematic Video Cuts
find_video_cuts(
    dataset_folder="../../data/",
    cuts_folder="../../data/cuts/"
)

In [ ]:
# Find Casual Video Cuts
find_video_cuts(
    dataset_folder="../../data/casual/",
    cuts_folder="../../data/casual/cuts/"
)